In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
NFLX_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/NFLX_predictions.csv", index_col=0)


In [3]:
NFLX_predictions.head()

,Date,Predictions
0,2019-07-31,0
1,2019-06-28,0
2,2019-05-31,1
3,2019-04-30,1
4,2019-03-29,0


In [4]:
NFLX_predictions.Date = pd.to_datetime(NFLX_predictions.Date, format="%Y-%m-%d")

In [5]:
NFLX_predictions['Predictions'] = NFLX_predictions['Predictions'].shift(-1)


In [6]:
NFLX_predictions = NFLX_predictions.dropna()


In [7]:
NFLX_predictions = NFLX_predictions[::-1]

In [8]:
NFLX_predictions.head(10)

,Date,Predictions
112,2010-03-31,1.0
111,2010-04-30,0.0
110,2010-05-28,1.0
109,2010-06-30,1.0
108,2010-07-30,1.0
107,2010-08-31,1.0
106,2010-09-30,0.0
105,2010-10-29,0.0
104,2010-11-30,0.0
103,2010-12-31,0.0


In [9]:
NFLX_monthly = pd.read_csv("/Users/fahad/project_repo/data/final/NFLX/NFLX_monthly.csv", index_col =0)

In [10]:
NFLX_monthly.columns = ['Open','High','Low','Close','Volume']

In [11]:
NFLX_predictions = NFLX_predictions.set_index("Date")

In [12]:
NFLX_monthly.head()

,Open,High,Low,Close,Volume
2019-08-15,324.25,328.58,288.00,295.76,76940768
2019-07-31,373.50,384.76,305.81,322.99,196641865
2019-06-28,343.56,375.00,332.65,367.32,111051945
2019-05-31,374.00,385.99,341.39,343.28,125299357
2019-04-30,359.00,384.80,342.27,370.54,165088190


In [13]:
NFLX_final_df = pd.merge(NFLX_predictions, NFLX_monthly, right_index=True, left_index=True)

In [14]:
NFLX_final_df.index = pd.to_datetime(NFLX_final_df.index, format ="%Y-%m-%d")

In [15]:
equity_dict = dict()

In [22]:
class Binary_Classification_1d(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.data.Predictions
        self.set_trailing_sl(4)
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
        
        if self.data.Predictions[-1] == 1.0 and (position.is_long !=True):
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif self.data.Predictions[-1] == 0.0 and (position.is_short !=True):
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.sell()
            
 

In [24]:
bt = Backtest(NFLX_final_df, Binary_Classification_1d, cash=10000, commission=.01)
bt.run()



Start                          2010-03-31 00:00:00
End                            2019-07-31 00:00:00
Duration                        3409 days 00:00:00
Exposure [%]                               97.3306
Equity Final [$]                       3.40939e+06
Equity Peak [$]                        3.87733e+06
Return [%]                                 33993.9
Buy & Hold Return [%]                      338.012
Max. Drawdown [%]                         -51.4876
Avg. Drawdown [%]                         -13.2008
Max. Drawdown Duration           578 days 00:00:00
Avg. Drawdown Duration           119 days 00:00:00
# Trades                                        39
Win Rate [%]                               76.9231
Best Trade [%]                             188.933
Worst Trade [%]                           -37.6861
Avg. Trade [%]                             20.5231
Max. Trade Duration              397 days 00:00:00
Avg. Trade Duration               86 days 00:00:00
Expectancy [%]                 

In [25]:
bt.plot(superimpose=False, plot_volume=False)

In [26]:
df = pd.DataFrame.from_dict([equity_dict])

In [27]:
df = df.T

In [28]:
df.columns = ['Equity']

In [29]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/NFLX_CNN_Equity.csv")

In [30]:
df

,Equity
2010-04-30,1.000000e+04
2010-05-28,8.792626e+03
2010-06-30,8.841282e+03
2010-07-30,8.344901e+03
2010-08-31,1.021406e+04
2010-09-30,1.319560e+04
2010-10-29,1.189430e+04
2010-11-30,9.289524e+03
2010-12-31,1.172269e+04
2011-01-31,8.630475e+03
